<div class="alert alert-block alert-success">
<h1><strong>NASA Data Fusion Analysis of Derechos and Their Impact on Rural America</h1>
</div>
    
Story authors: Madison Wallner<sup>1</sup>, Andrew Blackford<sup>1</sup>, Udaysankar Nair<sup>1</sup><br />
<sup>1</sup>The University of Alabama in Huntsville (UAH)

Notebook editor: Kyle Lesinger<sup>1</sup>

# Run This Notebook

[Launch in VEDA JupyterHub (requires access)](https://hub.openveda.cloud/hub/user-redirect/git-pull?repo=https://github.com/NASA-IMPACT/veda-docs&urlpath=lab/tree/veda-docs/user-guide/notebooks/stories/derechos.ipynb&branch=main) 


<div style="
text-align: center;
margin: 20px auto;
padding: 15px 25px;
background-color: #4CAF50; /* A nice green background /
color: white;
border-radius: 8px;
box-shadow: 0 4px 8px rgba(0,0,0,0.2);
display: inline-block; / To make div fit content /
cursor: pointer;
transition: background-color 0.3s ease, transform 0.2s ease;
">
<a href="https://hub.openveda.cloud/hub/user-redirect/git-pull?repo=https://github.com/NASA-IMPACT/veda-docs&urlpath=lab/tree/veda-docs/user-guide/notebooks/stories/derechos.ipynb&branch=main"
target="_blank"  / Opens in a new tab /
style="
text-decoration: none; / Remove underline /
color: white;
font-size: 1.2em; / Slightly larger text /
font-weight: bold;
display: flex;
align-items: center;
justify-content: center;
gap: 10px; / Space between icon and text */
"
onmouseover="this.parentNode.style.backgroundColor='#45a049'; this.parentNode.style.transform='translateY(-2px)';"
onmouseout="this.parentNode.style.backgroundColor='#4CAF50'; this.parentNode.style.transform='translateY(0)';">
🚀 Launch in VEDA JupyterHub (requires access)
</a>
</div>

<br>

Explanation of the HTML and CSS:

<div style="...">: This creates a container for your link, allowing us to style its background, padding, shadow, and center it.

text-align: center; margin: 20px auto;: Centers the block horizontally.

background-color, color, border-radius, box-shadow: For visual appeal.

display: inline-block;: Makes the div only as wide as its content, allowing margin: auto to center it.

cursor: pointer; transition;: Adds a hover effect for better user experience.

<a href="..." target="_blank" style="...">: This is your actual link.

target="_blank": Ensures the link opens in a new browser tab.

text-decoration: none; color: white;: Styles the link text.

font-size: 1.2em; font-weight: bold;: Makes the text larger and bold.

display: flex; align-items: center; justify-content: center; gap: 10px;: Uses flexbox to neatly align the rocket emoji and text horizontally with some spacing.

onmouseover, onmouseout: JavaScript inline events to create a subtle hover effect on the button-like div.

🚀: This is a Unicode rocket emoji. Emojis are widely supported and render directly in Jupyter Markdown/HTML.

To use this:

Copy and paste the entire HTML block above into a Markdown cell in your Jupyter Notebook/Lab.

Run the cell.

You should now see a much more prominent, styled button-like link with a rocket emoji, making it visually appealing and clear for users to launch the notebook.

<div class="alert alert-block" style="
     background-color: #f8d7da;
     color: #721c24;
     border-left: 4px solid #28a745;
  ">
Disclaimer: it is highly recommended to run a tutorial within NASA VEDA JupyterHub, which already includes functions for processing and visualizing data specific to VEDA stories. Running the tutorial outside of the VEDA JupyterHub may lead to errors, specifically related to EarthData authentication. Additionally, it is recommended to use the Pangeo workspace within the VEDA JupyterHub, since certain packages relevant to this tutorial are already installed.</div>

If you **do not** have a VEDA Jupyterhub Account you can launch this notebook using mybinder by clicking the icon below.

<a href="https://binder.openveda.cloud/v2/gh/NASA-IMPACT/veda-docs/9c8cdbae92906fb7062b8a0c759dad90e223a4f9?urlpath=lab%2Ftree%2Fuser-guide%2Fnotebooks%2Fstories%2Fderechos.ipynb">
<img src="https://binder.openveda.cloud/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> </a>

## Example scripts to process and visualize information

In [2]:
# Load libraries
import plotutils as putils
import earthaccess
from earthaccess import Auth, Store
import datetime
import pandas as pd
import xarray as xr
import fsspec
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets


In [ ]:
#For retrieving data already catalogued in VEDA STAC
STAC_API_URL="https://openveda.cloud/api/stac"
RASTER_API_URL = "https://openveda.cloud/api/raster"

# What is a Derecho?

A **derecho** is a long-lasting, fast-moving windstorm that is produced from a line of severe thunderstorms. For a storm to be called a derecho, there must be:

- A concentrated area of severe wind reports over 400 miles (650 km) in length and at least 60 miles (96 km) wide.
- Several wind gusts reported over 75 mph (120 km/h).
- Sustained winds of at least 58 mph (93 km/h). (SPC)

Unlike single thunderstorms, derechos form within mesoscale (i.e., mid-sized) convective systems (MCS)—large storm clusters that produce strong, long-lasting straight-line winds.

## Conditions that Help a Derecho Form Include:

- **Strong instability** (Convective Available Potential Energy (CAPE) over 2000–4000 J/kg): provides energy for strong updrafts and intense thunderstorms.  
- **High low-level moisture** (dewpoints of 65–75 °F): keeps storms going by supplying moisture.  
- **Strong mid- and upper-level winds** (wind shear over 40 knots): help organize storms and push them forward.  
- **A well-defined cold pool**: rain-cooled air at the surface strengthens the storm by increasing wind speeds at the front of the system.  


<div class="alert alert-block alert-warning">
<h1><strong>Example:</strong> Retrieve GLDAS data from EarthData portal</h1>
<h3>Global Land Data Assimilation System (GLDAS) data helps assess soil moisture levels before and after the storm, showing how pre-existing drought conditions contributed to dust transport and how heavy rainfall may have impacted runoff and flooding.</h3>
</div>


### Processing steps:
1.) Provide credentials for EarthAccess authentication<br />
2.) Select a start and end date for the data granules<br />
3.) Add <a href="https://search.earthdata.nasa.gov/search/granules?p=C1690022359-GES_DISC&pg[0][v]=f&pg[0][gsk]=-start_date&q=GLDAS_CLSM10_3H_2.1&tl=1347014600.484!5!!&lat=0.05719976537514171&long=-0.07464366545408296&zoom=3.0033333333333334"
        target="_blank" rel="noopener noreferrer"
         style="color: blue; text-decoration: none;">
      model name and version from EarthData portal
    </a><br />
4.) Add the prefix for the data within the bucket (e.g., remote sensing mission, reanalysis model, etc.)<br />
5.) Retrieve all files within the collection based on the start and end date. (If the same year is selected for start_date and end_date, then only the common year will be retrieved. Else all granules are retrived.<br />
6.) Filter all collected granules based on the start_ and end_date range.<br />
7.) Compute daily mean of hourly files<br />
8.) Plot variable based on coordinates selected<br />
9.) Create a .gif over the start_ and end_date range

In [ ]:
# ── Authenticate ──────────────────────────────────────────────────────────────
auth = Auth()
### If authentication comes from personal .netrc file
auth.login(strategy="netrc")
print(f"Authenticated: {auth.authenticated}")
## If authentication comes from local environment
auth.login(strategy="environment")
print(f"Authenticated: {auth.authenticated}")

store = Store(auth)

### Select dates and find Earthdata links

In [ ]:
# ── Search by date ────────(requires >2GB RAM if more than 40 dates are selected)───────────────
start_date = datetime.datetime(2022, 4, 1)
end_date = datetime.datetime(2022, 5, 12)
date_array = pd.date_range(start=start_date, end=end_date, freq='D').to_pydatetime()

# ── Dataset ─────────────────────────────────────────────────────────────────────────────────────
short_name="GLDAS_NOAH025_3H"
version="2.1"
variable = 'SoilMoi0_10cm_inst' #Only select a single variable of interest

# get all granules
print('Retrieving data granules from Earthaccess') 
results = earthaccess.search_data(
   short_name=short_name,
   version=version,
   temporal=(start_date, end_date),
   cloud_hosted=True
)

# grab the S3 URLs
urls = [g["umm"]["RelatedUrls"][1]["URL"] for g in results]
print(f"Found {len(urls)} files")

# ── Open granules ───────────────────────────────────────────────────────────────────────────────
granules=earthaccess.open(results)
fs = earthaccess.get_s3_filesystem(results=results)   # auto‐picks the right DAAC & creds

fsspec_caching = {
    "cache_type": "mmap"
}
lat_range=slice(24, 50)
lon_range=slice(-125, -66)

def subset(ds):
    return ds[[variable]].sel(lat=lat_range, lon=lon_range)

ds = xr.open_mfdataset(
    [fs.open(url, **fsspec_caching) for url in urls],
    chunks="auto",
    concat_dim="time",
    combine="nested",
    parallel=True,
    data_vars="minimal",
    coords="minimal",
    compat="override",
    join="exact",
    preprocess=subset,
)

# ── Compute Daily Mean ───────────────────────────────────────────────────────────────────────────────
ds_resampled = ds.resample(time="1D").mean(dim='time').compute()
ds_resampled

### Create interactive geospatial folium plot

In [ ]:
# Use the plot_folium_from_xarray function from plotutils
putils.plot_folium_from_xarray(dataset=ds_resampled['SoilMoi0_10cm_inst'],
                  day_select='2022-05-11',
                  bbox=[-130, 33, -90, 50],
                  var_name_for_title='Soil Moisture from GLDAS (m³/m³) [subset]',
                  matplot_ramp = 'YlOrRd_r',
                  zoom_level = 4.5,
                  flipud=False,
                  save_tif=False,
                  tif_filename=None,
                  crs = '4326',
                  opacity = 0.8)

### Create a gif over the date range

In [ ]:
# Usage:
putils.matplotlib_gif(data=ds_resampled['SoilMoi0_10cm_inst'],
               bbox= [-130, 33, -90, 50],
               gif_savename= "soil_matplotlib.gif",
               duration=2,
               cmap="YlOrRd_r")

<div class="alert alert-block alert-warning">
<h2><strong>Example:</strong> Pull WLDAS soil moisture data from the VEDA STAC catalog and visualize.</h2>

The Western Land Data Assimilation System (WLDAS) is a regional instance of NASA’s Land Information System (LIS), developed at Goddard Space Flight Center for the western United States.  It integrates meteorological forcings (precipitation, radiation, temperature, humidity, wind, surface pressure) and satellite-derived parameters (vegetation class, soil texture, elevation) into the Noah-MP land surface model using data assimilation techniques.

Soil moisture critically controls the partitioning of net surface energy into latent (evapotranspiration) versus sensible (heating) fluxes. Wetter soils favor latent heat, stabilizing the boundary layer, whereas drier soils boost sensible heating, enhancing near-surface temperature and convective available potential energy (CAPE). These processes govern where and when thunderstorms can initiate and organize.
</div>


### Processing steps:
1.) Choose STAC catalog ID and date<br />
2.) Retrieve collection information and items from VEDA STAC catalog<br />
3.) Retrieve item statistics and tiling information<br />
4.) Plot data<br />

### Choose variable and retrieve json from VEDA STAC catalogue

In [ ]:
#TODO: Change collection_ID and date

collection_id = 'wldas-derecho-sm'
date = "2022-05-11"

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────
url = f"{STAC_API_URL.rstrip('/')}/collections/{collection_id}"

response = requests.get(url)
response.raise_for_status()
collection = response.json()

# grab the dashboard render block
dashboard_rend = collection["renders"]["dashboard"]

bands      = dashboard_rend["bidx"]           
asset_keys = dashboard_rend["assets"]         
(vmin, vmax), = dashboard_rend["rescale"]    
cmap_name  = dashboard_rend["colormap_name"]

# ── VEDA Item Request ─────────────────────────────────────────────────────────────────────────────────────
url = f"{STAC_API_URL.rstrip('/')}/search"
payload = {
    "collections": [collection_id],
    "query": {
        "datetime": {
            "eq": f"{date}T00:00:00"
        }
    },
    "limit": 10,
}

response = requests.post(url, json=payload)
response.raise_for_status()
data = response.json()
items = data.get("features")

if items is None:
    raise KeyError(f"No 'features' key in search response: {data.keys()}")

print(f"\nFound {len(items)} features in collection {collection_id} on date {date}")

item = items[0] #only return the first item


### Retrieve tiling information

In [ ]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
assets = "cog_default"
colormap_name = "rdbu_r"
color_formula = "gamma+r+1.05"  ## DO NOT CHANGE
rescale = (vmin,vmax)

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item['id']}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula={color_formula}&colormap_name={colormap_name}"
    f"&rescale={rescale[0]},{rescale[1]}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)


### Plot data

In [ ]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles['tiles'][0],
    center_coords=[41.5, -110],
    zoom_level=4,
    rescale=rescale,
    colormap_name=colormap_name,
    layer_name="WLDAS soil moisture",
    date=f'{date}T00:00:00Z',
    colorbar_caption="Soil Moisture (m3/m3)",
    attribution='VEDA WLDAS Soil Moisture',
    tile_name='WLDAS Soil Moisture Data',
    opacity=0.8,
    capitalize_cmap=True
)

# Display the map
m

<div class="alert alert-block alert-warning">
<h2><strong>Example:</strong> Retrieve MERRA-2 hourly files for Aerosol Optical Thickness</h2>

Aerosol Optical Thickness (AOT), also called Aerosol Optical Depth (AOD), is a dimensionless measure of how much sunlight aerosols—tiny particles like dust, smoke or sea salt—scatter and absorb as it travels through a column of atmosphere. In practical terms, an AOT of 0.1 means only 10 % of the direct solar beam is extinguished by aerosols before reaching the surface.

The intense straight-line winds in derechos can uplift large quantities of soil and dust, dramatically increasing AOT downwind. Tracking AOT in near-real-time reveals the spatial extent and intensity of these airborne dust plumes.
</div>


### Processing steps:
1.) Select a start and end date for the data granules<br />
2.) Add <a href="https://search.earthdata.nasa.gov/search/granules/collection-details?p=C1276812830-GES_DISC&pg[0][v]=f&pg[0][gsk]=-start_date&q=merra%20aerosol&tl=1031468010.052!5!!"
        target="_blank" rel="noopener noreferrer"
         style="color: blue; text-decoration: none;">
      model name and version from EarthData portal
    </a><br />
3.) Add the prefix for the data within the bucket (e.g., remote sensing mission, reanalysis model, etc.)<br />
4.) Retrive all files within the collection based on the start and end date. (If the same year is selected for start_date and end_date, then only the common year will be retrieved. Else all granules are retrived.<br />
5.) Filter all collected granules based on the start_ and end_date range.<br />
6.) Plot variable based on coordinates selected<br />
7.) Create a .gif over the start_ and end_date range

### Select and filter

In [ ]:
# ── Search by date ──────────────────────────────────────────────────────────────────────────────
start_date = datetime.datetime(2022, 5, 12)
end_date = datetime.datetime(2022, 5, 12)
date_array = pd.date_range(start=start_date, end=end_date, freq='D').to_pydatetime()

# ── Dataset ─────────────────────────────────────────────────────────────────────────────────────
short_name="M2T1NXAER"
version="5.12.4"
variable = 'TOTEXTTAU' #Total aerosol extinction optical thickness

# get all granules
print('Retrieving data granules from Earthaccess') 
results = earthaccess.search_data(
   short_name=short_name,
   version=version,
   temporal=(start_date, end_date),
   cloud_hosted=True
)

# grab the S3 URLs
urls = [g["umm"]["RelatedUrls"][1]["URL"] for g in results]
print(f"Found {len(urls)} files")

# ── Open granules ───────────────────────────────────────────────────────────────────────────────
granules=earthaccess.open(results)
fs = earthaccess.get_s3_filesystem(results=results)   # auto‐picks the right DAAC & creds

fsspec_caching = {
    "cache_type": "mmap"
}
lat_range=slice(24, 50)
lon_range=slice(-125, -66)

def subset(ds):
    return ds[[variable]].sel(lat=lat_range, lon=lon_range)

ds = xr.open_mfdataset(
    [fs.open(url, **fsspec_caching) for url in urls],
    chunks="auto",
    concat_dim="time",
    combine="nested",
    parallel=True,
    data_vars="minimal",
    coords="minimal",
    compat="override",
    join="exact",
    preprocess=subset,
)

ds

## Open the MERRA file and plot by hour in a .gif for the selected variable AOT

In [ ]:
# Use the matplotlib_gif function from plotutils
putils.matplotlib_gif(data=ds['TOTEXTTAU'],
               bbox= [-130, 33, -90, 50],
               gif_savename= "merra_aot.gif",
               duration=2,
               cmap="YlOrRd_r")

<div class="alert alert-block alert-warning">
<h2><strong>Example:</strong> Pull MODIS Aerosol Optical Depth data from the VEDA STAC catalog and visualize</h2>
</div>


In [ ]:
collection_id = 'modis-derecho'
date = "2022-05-12"

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────
url = f"{STAC_API_URL.rstrip('/')}/collections/{collection_id}"

response = requests.get(url)
response.raise_for_status()
collection = response.json()

# grab the dashboard render block
dashboard_rend = collection["renders"]["dashboard"]

bands      = dashboard_rend["bidx"]           
asset_keys = dashboard_rend["assets"]         
(vmin, vmax), = dashboard_rend["rescale"]    
cmap_name  = dashboard_rend["colormap_name"]

# ── VEDA Item Request ─────────────────────────────────────────────────────────────────────────────────────
url = f"{STAC_API_URL.rstrip('/')}/search"
payload = {
    "collections": [collection_id],
    "query": {
        "datetime": {
            "eq": f"{date}T00:00:00"
        }
    },
    "limit": 10,
}

response = requests.post(url, json=payload)
response.raise_for_status()
data = response.json()
items = data.get("features")

if items is None:
    raise KeyError(f"No 'features' key in search response: {data.keys()}")

print(f"\nFound {len(items)} features in collection {collection_id} on date {date}")

item = items[0] #only return the first item

# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
assets = "cog_default"
colormap_name = "viridis"
color_formula = "gamma+r+1.05"  ## DO NOT CHANGE
rescale = (vmin,vmax)

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item['id']}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula={color_formula}&colormap_name={colormap_name}"
    f"&rescale={rescale[0]},{rescale[1]}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

In [ ]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles['tiles'][0],
    center_coords=[43.5, -96],
    zoom_level=6,
    rescale=rescale,
    colormap_name=colormap_name,
    layer_name="MODIS AOD",
    date=f'{date}T00:00:00Z',
    colorbar_caption="Aerosol Optical Depth",
    attribution='VEDA MODIS AOD',
    tile_name='MODIS AOD',
    opacity=0.8
)

# Display the map
m

<div class="alert alert-block alert-warning">
<h2><strong>Example:</strong> Pull NCEI interpolated wind gusts data from the VEDA STAC catalog and visualize.</h2>
    
The NCEI Interpolated Wind Gusts product takes the discrete station‐reported peak wind gusts from the NCEI Storm Events Database—a standardized archive of severe‐weather observations dating back to 1950—and uses spatial interpolation to generate a continuous gridded field of maximum gust speeds across the derecho swath. 

By filling in the gaps between point measurements, it reveals the full geographic extent and intensity gradients of the derecho’s outflow winds, often uncovering zones of extreme gust (e.g., 70 mph +) that lie between and beyond individual station sites. 

</div>


In [ ]:
collection_id = 'windgusts-derecho'
date = "2022-05-12"

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────
url = f"{STAC_API_URL.rstrip('/')}/collections/{collection_id}"

response = requests.get(url)
response.raise_for_status()
collection = response.json()

# grab the dashboard render block
dashboard_rend = collection["renders"]["dashboard"]

bands      = dashboard_rend["bidx"]           
asset_keys = dashboard_rend["assets"]         
(vmin, vmax), = dashboard_rend["rescale"]    
cmap_name  = dashboard_rend["colormap_name"]

# ── VEDA Item Request ─────────────────────────────────────────────────────────────────────────────────────
url = f"{STAC_API_URL.rstrip('/')}/search"
payload = {
    "collections": [collection_id],
    "query": {
        "datetime": {
            "eq": f"{date}T00:00:00"
        }
    },
    "limit": 10,
}

response = requests.post(url, json=payload)
response.raise_for_status()
data = response.json()
items = data.get("features")

if items is None:
    raise KeyError(f"No 'features' key in search response: {data.keys()}")

print(f"\nFound {len(items)} features in collection {collection_id} on date {date}")

item = items[0] #only return the first item

# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
assets = "cog_default"
colormap_name = "viridis"
color_formula = "gamma+r+1.05"  ## DO NOT CHANGE
rescale = (vmin,vmax)

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item['id']}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula={color_formula}&colormap_name={colormap_name}"
    f"&rescale={rescale[0]},{rescale[1]}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

In [ ]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles['tiles'][0],
    center_coords=[45, -97.7],
    zoom_level=6,
    rescale=rescale,
    colormap_name=colormap_name,
    layer_name="Wind Gusts",
    date=f'{date}T00:00:00Z',
    colorbar_caption="Wind Gusts (m/s)",
    attribution='VEDA Wind Gusts',
    tile_name='Wind Gusts',
    opacity=0.8
)

# Display the map
m

<div class="alert alert-block alert-warning">
<h2><strong>Example:</strong> Pull Black Marble Night Lights data from the VEDA STAC catalog and visualize.</h2>

During extreme windstorms such as derechos, sudden drops in nighttime brightness in the Black Marble imagery reveal where power infrastructure has failed and outages have occurred, while the re-illumination of areas over subsequent days tracks the pace and extent of electrical service restoration. This makes Black Marble a powerful, near-real-time proxy for assessing societal impacts and grid resilience in the storm’s wake.
</div>


In [ ]:
collection_id = 'nightlights-derecho'
date = "2022-05-10"

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────
url = f"{STAC_API_URL.rstrip('/')}/collections/{collection_id}"

response = requests.get(url)
response.raise_for_status()
collection = response.json()

# grab the dashboard render block
dashboard_rend = collection["renders"]["dashboard"]

bands      = dashboard_rend["bidx"]           
asset_keys = dashboard_rend["assets"]         
(vmin, vmax), = dashboard_rend["rescale"]    
cmap_name  = dashboard_rend["colormap_name"]

# ── VEDA Item Request ─────────────────────────────────────────────────────────────────────────────────────
url = f"{STAC_API_URL.rstrip('/')}/search"
payload = {
    "collections": [collection_id],
    "query": {
        "datetime": {
            "eq": f"{date}T00:00:00"
        }
    },
    "limit": 10,
}

response = requests.post(url, json=payload)
response.raise_for_status()
data = response.json()
items = data.get("features")

if items is None:
    raise KeyError(f"No 'features' key in search response: {data.keys()}")

print(f"\nFound {len(items)} features in collection {collection_id} on date {date}")

item = items[0] #only return the first item

# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
assets = "cog_default"
colormap_name = "bwr"
color_formula = "gamma+r+1.05"  ## DO NOT CHANGE
rescale = (vmin,vmax)

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item['id']}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula={color_formula}&colormap_name={colormap_name}"
    f"&rescale={rescale[0]},{rescale[1]}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

In [ ]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles['tiles'][0],
    center_coords=[45, -93],
    zoom_level=8,
    rescale=rescale,
    colormap_name=colormap_name,
    layer_name="Black Marble Night Lights",
    date=f'{date}T00:00:00Z',
    colorbar_caption="Artificial Light",
    attribution='VEDA Black Marble Night Lights',
    tile_name='Black Marble Night Lights',
    opacity=0.8
)

print('Visualization of St. Paul, Minnesota where values approaching 255 indicate power outages.')
# Display the map
m

## The economic impact reached beyond the crop fields. Many grain silos, irrigation systems, and farm buildings were also damaged or destroyed, adding to the financial burden on farmers.  See image below to identify crop types across the Midwest and the Storm Prediction Center reports across the Midwest.

### Use the slider below to investigate the crop-land type and the extreme weather reports across the Midwest.

In [ ]:
# Usage example:
# Replace these with your actual image paths
img1_path = "images/derecho/Derecho-Crop.jpg"
img2_path = "images/derecho/Derecho-storm-reports.jpg"

putils.create_animated_blend(img1_path, img2_path)(img1_path, img2_path)

<div class="alert alert-block alert-danger">
<h2><strong>End of visualizations</strong>
</div>


In [ ]:
# Remove created .gifs and core files from EarthAccess
import os
import glob

# find all .gif files in the current directory
for gif_path in glob.glob("*.gif"):
    try:
        os.remove(gif_path)
        print(f"Removed {gif_path}")
    except OSError as e:
        print(f"Error removing {gif_path}: {e}")

# find all .gif files in the current directory
for core_path in glob.glob("core.*"):
    try:
        os.remove(core_path)
        print(f"Removed {core_path}")
    except OSError as e:
        print(f"Error removing {core_path}: {e}")
